In [14]:
import numpy as np
import re
import porter as p
import TextRepresenter as tr
from collections import Counter
pt=tr.PorterStemmer()

from Parser import Parser
from IndexerSimple import IndexerSimple

# Exercice 1

In [5]:
docs=["the new home has been saled on top forecasts",
     "the home sales rise in july",
     "there is an increase in home sales in july",
     "july encounter a new home sales rise"]

stopWords=["the","a","an","on","behind","under","there","in"]

## 1)

In [39]:
#en minuscule
tmp1=str.lower(docs[1])
#sépare les mots
tmp2=tmp1.split()
#enlève les stopWords
tmp3=[word for word in tmp2 if word not in stopWords]
#applique la radicalisation sur chaque mot
tmp4=map(p.stem, tmp3)
#coompte les éléments et convertit en dictionnaire
dict(Counter(tmp4))

{'home': 1, 'sale': 1, 'rise': 1, 'juli': 1}

## 2)

In [6]:
index = {}
for i in range(len(docs)):
    index[i] = dict(Counter(map(p.stem, [word for word in (str.lower(docs[i])).split() if word not in stopWords])))
index

{0: {'new': 1,
  'home': 1,
  'ha': 1,
  'been': 1,
  'sale': 1,
  'top': 1,
  'forecast': 1},
 1: {'home': 1, 'sale': 1, 'rise': 1, 'juli': 1},
 2: {'is': 1, 'increas': 1, 'home': 1, 'sale': 1, 'juli': 1},
 3: {'juli': 1, 'encount': 1, 'new': 1, 'home': 1, 'sale': 1, 'rise': 1}}

In [7]:
indexInverse = {}
for numDoc in index:
    for word in index[numDoc]:
        if(word not in indexInverse):
            indexInverse[word]= {}
        if(numDoc not in indexInverse[word]):
            indexInverse[word][numDoc]=1
        else:
            indexInverse[word][numDoc]+=1
indexInverse

{'new': {0: 1, 3: 1},
 'home': {0: 1, 1: 1, 2: 1, 3: 1},
 'ha': {0: 1},
 'been': {0: 1},
 'sale': {0: 1, 1: 1, 2: 1, 3: 1},
 'top': {0: 1},
 'forecast': {0: 1},
 'rise': {1: 1, 3: 1},
 'juli': {1: 1, 2: 1, 3: 1},
 'is': {2: 1},
 'increas': {2: 1},
 'encount': {3: 1}}

## 3)

In [37]:
df={}
for word in indexInverse:
    df[word]=len(indexInverse[word])
df

{'new': 2,
 'home': 4,
 'ha': 1,
 'been': 1,
 'sale': 4,
 'top': 1,
 'forecast': 1,
 'rise': 2,
 'juli': 3,
 'is': 1,
 'increas': 1,
 'encount': 1}

In [38]:
tf_idf = {}
for numDoc in index:
    tf_idf[numDoc] = {}
    for word in index[numDoc]:
        tf_idf[numDoc][word] = index[numDoc][word] * np.log( (1+len(index)) / (1+df[word]) )
        
tf_idf

{0: {'new': 0.5108256237659907,
  'home': 0.0,
  'ha': 0.9162907318741551,
  'been': 0.9162907318741551,
  'sale': 0.0,
  'top': 0.9162907318741551,
  'forecast': 0.9162907318741551},
 1: {'home': 0.0,
  'sale': 0.0,
  'rise': 0.5108256237659907,
  'juli': 0.22314355131420976},
 2: {'is': 0.9162907318741551,
  'increas': 0.9162907318741551,
  'home': 0.0,
  'sale': 0.0,
  'juli': 0.22314355131420976},
 3: {'juli': 0.22314355131420976,
  'encount': 0.9162907318741551,
  'new': 0.5108256237659907,
  'home': 0.0,
  'sale': 0.0,
  'rise': 0.5108256237659907}}

# Exercice 2

## 1)

In [65]:
def buildDocCollectionSimple(chemin):
    
    res = {}
    currentI = None
    isInT = False
    
    file = open(chemin, 'r') 
    
    while True:
        #lis une seule ligne
        line = file.readline()
        
        #si ligne vide, fin du fichier
        if not line:
            break
            
        #récupère la ligne sous forme de mots
        words=line.split()
        
        #Si la ligne n'est pas vide
        if(len(words)>0):
            #si c'est une balise I
            if(words[0]==".I"):
                res[words[1]] = "" #Je créé le dictionnaire pour ce document
                currentI = words[1] #Je garde l'identifiant du document courant
                isInT=False #J'indique que je ne suis pas danc une balise T
            #Sinon si c'est une balise T
            elif(words[0]==".T"):
                isInT=True #J'indique que je suis danc une balise T
            else:
                #Sinon si c'est une autre balise
                if(words[0][0]=='.'):
                    isInT=False #J'indique que je ne suis pas danc une balise T
                else:
                    #Sinon si on est dans .I et dans .T
                    if(isInT==True and currentI != None):
                        res[currentI] += line #J'ajoute la ligne courante au texte du document courant
        
        
    file.close()
    
    return res

In [64]:
buildDocCollectionSimple('cacmShort-good.txt')

{'1': 'Preliminary Report-International Algebraic Language\njgkfldjgfkld\n',
 '2': 'Extraction of Roots by Repeated Subtractions for Digital Computers\n',
 '3': 'Techniques Department on Matrix Program Schemes\n',
 '4': 'Glossary of Computer Engineering and Programming Terminology\n',
 '5': 'Two Square-Root Approximations\n',
 '6': 'The Use of Computers in Inspection Procedures\n',
 '7': 'Glossary of Computer Engineering and Programming Terminology\n',
 '8': 'On The Equivalence and Transformation of Program Schemes\n',
 '9': 'Proposal for an UNCOL\n',
 '10': 'Glossary of Computer Engineering and Programming Terminology\n',
 '11': 'The Problem of Programming Communication with\nChanging Machines A Proposed Solution-Part 2\n'}

## 2)

In [100]:
def buildDocumentCollectionRegex(chemin):
    
    file = open(chemin, 'r') 
    
    lines = file.read()
    #regex = re.compile(r'.I (\w+?)\n(?:\.T\n([\w\d\s-]+))?')
    regex = re.compile(r'\.I (\d+)\n(?:\.T\n([\w\s.-]+?)\n\.)?')
    res = dict(regex.findall(lines))
    
    file.close()
    
    return res

In [101]:
buildDocumentCollectionRegex('cacmShort-good.txt')

{'1': 'Preliminary Report-International Algebraic Language\njgkfldjgfkld',
 '2': 'Extraction of Roots by Repeated Subtractions for Digital Computers',
 '3': 'Techniques Department on Matrix Program Schemes',
 '4': 'Glossary of Computer Engineering and Programming Terminology',
 '5': 'Two Square-Root Approximations',
 '6': 'The Use of Computers in Inspection Procedures',
 '7': 'Glossary of Computer Engineering and Programming Terminology',
 '8': 'On The Equivalence and Transformation of Program Schemes',
 '9': 'Proposal for an UNCOL',
 '10': 'Glossary of Computer Engineering and Programming Terminology',
 '11': 'The Problem of Programming Communication with\nChanging Machines A Proposed Solution-Part 2'}

# Exercice 3

## 1)

In [ ]:
#CF Document.py

## 2)

In [ ]:
#CF Parser.py

Normalement, une classe parser par collection (veut dire? un parserCamc, un parserCisi ?)
<br/><br/>
La fonction parse ci dessous ne vériie pas que chaque document a un identifiant unique (peut etre vérifié en utilisant un set d'identifiant commun aux instances par ex)

Pour parser, voici carac balise:
- titre = plusieurs lignes de texte
- texte = plusieurs lignes de textes
- date = 4 digit ou texte puis 4 digit (donc regex sur 4 digit)
- auteur = nom, prénom1 prenom2 etc (les prénoms sont espacé et peuvent etre de la forme A.). Un auteur par ligne. Pas forcément d'espace entre nom "," et prénom
- mots = plusieurs mots/expressions séparés par une virgule sur plusieurs ligne (mot1 mot2, mot3, mot4 mot5 mot6)
- liens = plusieurs ligne et sur chaque ligne: number1    number2    number3 (3 nombres séparé par 'tab') mais seul premier intéressant (id du truc cité ou du truc qui cite)

## 3) 4) 5)

In [ ]:
#CF InderxerSimple.py

## 6)
TODO : mettre les 3 lignes de codes suivante dans fichiers Test.py

In [4]:
collec1 = Parser.parseCacmCisi('cisi/cisi.txt')
indexer=IndexerSimple(collec1)
indexer.indexation()